In [15]:
from sage import EBSD
ebsdfile = EBSD("ebsd.ctf")
# ebsdfile.plot()

In [16]:
ebsdfile.phases()

{'NaN': 19.208213233621212,
 'Forsterite': 48.151226402053766,
 'Enstatite  Opx AV77': 18.753008655785035,
 'Diopside   CaMgSi2O6': 12.200015167659108,
 'Anorthite': 0.05246826758977829,
 'Hornblende': 0.08390619932347938,
 'Chromite': 1.5149237394841597,
 'Pyrope': 0.03623833448345784}

In [17]:
df = ebsdfile.get_ebsd_data()

In [18]:
df

,Phase,X,Y,Bands,Error,Euler1,Euler2,Euler3,MAD,BC,BS
0,0,0.0,0.0,0,5,360.0,180.0,0.0,0.0,0,0
1,0,10.0,0.0,0,5,360.0,180.0,0.0,0.0,0,0
2,0,20.0,0.0,0,5,360.0,180.0,0.0,0.0,0,0
3,0,30.0,0.0,0,5,360.0,180.0,0.0,0.0,0,0
4,0,40.0,0.0,0,5,360.0,180.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7436871,0,35330.0,21010.0,0,5,360.0,180.0,0.0,0.0,0,0
7436872,0,35340.0,21010.0,0,5,360.0,180.0,0.0,0.0,0,0
7436873,0,35350.0,21010.0,0,5,360.0,180.0,0.0,0.0,0,0
7436874,0,35360.0,21010.0,0,5,360.0,180.0,0.0,0.0,0,0


In [19]:
df = df[df['Phase'] != 0]
downsampling_factor = 20
df = df.iloc[::downsampling_factor]

In [20]:
df

,Phase,X,Y,Bands,Error,Euler1,Euler2,Euler3,MAD,BC,BS
3446,1,34460.0,0.0,10,0,32.0197,35.7966,333.1715,0.7780,93,61
3466,1,34660.0,0.0,11,0,31.6918,36.0418,333.9823,0.7369,124,81
3486,1,34860.0,0.0,10,0,31.3562,36.4128,334.1218,0.9416,145,82
3514,2,35140.0,0.0,9,0,87.4642,92.8881,189.4273,0.9671,86,56
6987,1,34490.0,10.0,10,0,32.6970,35.6833,332.7937,1.0525,121,72
...,...,...,...,...,...,...,...,...,...,...,...
7429874,1,740.0,21000.0,12,0,97.7427,115.8920,305.1532,0.6564,149,77
7433340,1,20.0,21010.0,11,0,168.5583,168.9959,195.8476,0.6327,121,64
7433382,1,440.0,21010.0,9,0,170.0603,169.1712,197.4050,0.6201,61,45
7433404,1,660.0,21010.0,8,0,97.0631,115.6784,305.1185,0.7973,52,44


In [21]:
import numpy as np
from scipy.spatial.transform import Rotation

euler_angles1 = np.array([70, 20, 10]) * np.pi / 180.0
euler_angles2 = np.array([40, 30, 10]) * np.pi / 180.0
quat1 = Rotation.from_euler('xyz', euler_angles1).as_quat()
quat2 = Rotation.from_euler('xyz', euler_angles2).as_quat()
mori_quat = Rotation.from_quat(quat1).inv() * Rotation.from_quat(quat2)
axis_angle = mori_quat.as_rotvec()
misorientation_angle_deg = np.linalg.norm(axis_angle) * 180.0 / np.pi

print("Misorientation angle between o1 and o2:", misorientation_angle_deg, "degrees")


Misorientation angle between o1 and o2: 31.586448302718836 degrees


In [22]:
import numpy as np
from math import acos, degrees

def euler_to_quaternion(phi, theta, psi):
    """
    Convert Euler angles (in radians) to a quaternion.
    """
    c1 = np.cos(phi / 2)
    s1 = np.sin(phi / 2)
    c2 = np.cos(theta / 2)
    s2 = np.sin(theta / 2)
    c3 = np.cos(psi / 2)
    s3 = np.sin(psi / 2)

    q0 = c1 * c2 * c3 + s1 * s2 * s3
    q1 = s1 * c2 * c3 - c1 * s2 * s3
    q2 = c1 * s2 * c3 + s1 * c2 * s3
    q3 = c1 * c2 * s3 - s1 * s2 * c3

    return np.array([q0, q1, q2, q3])

def quaternion_to_angle(q1, q2):
    """
    Calculate the misorientation angle between two quaternions.
    """
    dot_product = np.dot(q1, q2)
    angle_radians = 2 * acos(abs(dot_product))
    return degrees(angle_radians)

# Define Euler angles for orientations o1 and o2
phi1, theta1, psi1 = 60 * np.pi / 180, 30 * np.pi / 180, 30 * np.pi / 180
phi2, theta2, psi2 = 30 * np.pi / 180, 10 * np.pi / 180, 30 * np.pi / 180

# Convert Euler angles to quaternions
q1 = euler_to_quaternion(phi1, theta1, psi1)
q2 = euler_to_quaternion(phi2, theta2, psi2)

# Calculate misorientation angle
misorientation_angle_degrees = quaternion_to_angle(q1, q2)

print(misorientation_angle_degrees)


35.927720259662685


In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

def euler_to_quaternion(phi, theta, psi):
    """
    Convert Euler angles (in radians) to a quaternion.
    """
    c1 = np.cos(phi / 2)
    s1 = np.sin(phi / 2)
    c2 = np.cos(theta / 2)
    s2 = np.sin(theta / 2)
    c3 = np.cos(psi / 2)
    s3 = np.sin(psi / 2)

    q0 = c1 * c2 * c3 + s1 * s2 * s3
    q1 = s1 * c2 * c3 - c1 * s2 * s3
    q2 = c1 * s2 * c3 + s1 * c2 * s3
    q3 = c1 * c2 * s3 - s1 * s2 * c3

    return np.array([q0, q1, q2, q3])

def quaternion_to_angle(q1, q2):
    """
    Calculate the misorientation angle between two quaternions.
    """
    dot_product = np.dot(q1, q2)
    angle_radians = 2 * np.arccos(np.abs(dot_product))
    return np.degrees(angle_radians)

def misorientation_angle(euler1, euler2):
    """
    Calculate the misorientation angle between two sets of Euler angles.
    """
    q1 = euler_to_quaternion(*np.radians(euler1))
    q2 = euler_to_quaternion(*np.radians(euler2))
    return quaternion_to_angle(q1, q2)

def assign_to_grains_parallel(df, threshold):
    grains = []
    grain_indices = {}

    total_rows = len(df)
    processed_rows = 0

    def process_row(row):
        nonlocal grains, grain_indices, processed_rows
        euler_angles = row[['Euler1', 'Euler2', 'Euler3']].values
        assigned = False
        for grain_idx, grain_euler in enumerate(grains):
            misorientation = misorientation_angle(euler_angles, grain_euler)
            if misorientation <= threshold:
                grain_indices[row.name] = grain_idx
                grains[grain_idx] = np.mean([grain_euler, euler_angles], axis=0)
                assigned = True
                break
        if not assigned:
            grain_indices[row.name] = len(grains)
            grains.append(euler_angles)

        processed_rows += 1
        progress = (processed_rows / total_rows) * 100
        print(f"Progress: {progress:.2f}%\r", end='')

    Parallel(n_jobs=-1, prefer="threads")(delayed(process_row)(row) for _, row in df.iterrows())

    return grain_indices

# Sample phase names
phase_names = ['nothn', 'Forsterite', 'Enstatite Opx AV77', 'Diopside CaMgSi2O6',  
               'Anorthite', 'Hornblende', 'Chromite', 'Pyrope']

# Assign rows to grains in parallel
threshold = 20 # Misorientation threshold
grain_indices = assign_to_grains_parallel(df[['Euler1', 'Euler2', 'Euler3']], threshold)

# Add grain indices to the dataframe
df['Grain'] = pd.Series(grain_indices)

# Display grains with their respective phase names
for grain_idx, group in df.groupby('Grain'):
    phase_counts = group['Phase'].value_counts()
    dominant_phase = phase_counts.idxmax()
    print(f"Grain {grain_idx}: Dominant Phase - {phase_names[dominant_phase]}, Size - {len(group)}")

In [ ]:
df

In [ ]:
# Define the minimum grain size threshold
min_grain_size = 100

# Filter out grains below the minimum size threshold
valid_grains = df['Grain'].value_counts()[df['Grain'].value_counts() >= min_grain_size].index

# Filter the dataframe to include only rows corresponding to valid grains
df_filtered = df[df['Grain'].isin(valid_grains)]

# Display grains with their respective phase names
for grain_idx, group in df_filtered.groupby('Grain'):
    phase_counts = group['Phase'].value_counts()
    dominant_phase = phase_counts.idxmax()
    print(f"Grain {grain_idx}: Dominant Phase - {phase_names[dominant_phase]}, Size - {len(group)}")


In [ ]:
import plotly.express as px

# Sample DataFrame (replace with your data)
df = df_filtered

# Create a scatter plot using Plotly
fig = px.scatter(df, x='X', y='Y', color='Phase', color_continuous_scale='viridis')

# Customize the layout
fig.update_layout(
    title="2D Scatter Plot with Phase",
    xaxis_title="X",
    yaxis_title="Y",
    coloraxis_colorbar_title="Phase",
    showlegend=True
)

# Show the plot
fig.show()


In [ ]:
import plotly.express as px

# Sample DataFrame (replace with your data)
df = df

# Create a scatter plot using Plotly
fig = px.scatter(df, x='X', y='Y', color='Phase', color_continuous_scale='viridis')

# Customize the layout
fig.update_layout(
    title="2D Scatter Plot with Phase",
    xaxis_title="X",
    yaxis_title="Y",
    coloraxis_colorbar_title="Phase",
    showlegend=True
)

# Show the plot
fig.show()


In [ ]:
df_filtered